# infos

In [ ]:
# Version 1 finale
# Notebook de construction des .csv

### Changelogs :

v1 :
- Construction notebook

# Preambule

<span style="color: #FF0000">S'assurer que l'intégralité des images soient disponibles dans un dossier.</span>


- Split de mushroom_observation : <span style="color: #26B260">(La colonne 'image_lien' sert de clé commune pour les dataframes construits)</span>
    - _mushroom_observation_cleaned_ Fichier de données original après traitements
    - _mushroom_observation_cleaned_sorted_ranks_ Fichier de données traité et suppression des ranks non utilisés après analyse
    - _mushroom_observation_key_ Contient uniquement les variables 'Key' pour analyses
    - _model_datas_10_ Pour l'entrainement du modèle sans le tri des images
    - _model_datas_10_filtered_ Pour l'entrainement du modèle avec tri des images

    <span style="color: #26B260">_Particularité :_</span> Le fichier 'img_supp.csv' contient la liste des images supprimées du jeu de données original. Ces images comportaient des photos hors sujet (ciel, nature, analyses microscopiques) qui n'entrent pas dans le cadre du projet.

# Librairies

In [5]:
import pandas as pd
import os

# Afficher toutes les colonnes des DF dans les cellules
pd.set_option('display.max_columns', None)

# Traitement de observation_mushroom.csv

<span style="color: #FF0000">Fichier CSV d'origine fourni avec les images</span>

In [52]:
# URL du fichier
lien_csv = '../data/observations_mushroom.csv'

# Construction d'un DataFrame en important le fichier
mushroom_observation = pd.read_csv(lien_csv, low_memory=False)

In [53]:
print("Nombre d'obervations initiales :", mushroom_observation.shape[0])
mushroom_observation.head(5)

Nombre d'obervations initiales : 647623


,image_lien,image_id,observation,label,image_url,user,date,gbif_info/kingdom,gbif_info/family,gbif_info/speciesKey,gbif_info/rank,gbif_info/phylum,gbif_info/orderKey,gbif_info/species,gbif_info/confidence,gbif_info/classKey,gbif_info/matchType,gbif_info/familyKey,gbif_info/status,gbif_info/usageKey,gbif_info/kingdomKey,gbif_info/genusKey,gbif_info/canonicalName,gbif_info/phylumKey,gbif_info/class,gbif_info/synonym,gbif_info/scientificName,gbif_info/genus,gbif_info/order,thumbnail,location,gbif_info/note,gbif_info
0,1.jpg,1,1,Xylaria polymorpha,http://mushroomobserver.org/images/320/1,1,2006-05-21 07:17:05,Fungi,Xylariaceae,5255104.0,SPECIES,Ascomycota,1298.0,Xylaria polymorpha,98.0,320.0,EXACT,4168.0,ACCEPTED,5255104.0,5.0,2568748.0,Xylaria polymorpha,95.0,Sordariomycetes,False,"Xylaria polymorpha (Pers.) Grev., 1824",Xylaria,Xylariales,1,214.0,NaN,NaN
1,2.jpg,2,2,Xylaria magnoliae,http://mushroomobserver.org/images/320/2,1,2006-05-21 07:17:06,Fungi,Xylariaceae,3461845.0,SPECIES,Ascomycota,1298.0,Xylaria magnoliae,98.0,320.0,EXACT,4168.0,ACCEPTED,3461845.0,5.0,2568748.0,Xylaria magnoliae,95.0,Sordariomycetes,False,"Xylaria magnoliae J.D. Rogers, 1979",Xylaria,Xylariales,1,53.0,NaN,NaN
2,3.jpg,3,3,Xylaria hypoxylon,http://mushroomobserver.org/images/320/3,1,2006-05-21 07:17:08,Fungi,Xylariaceae,8631710.0,SPECIES,Ascomycota,1298.0,Xylaria hypoxylon,98.0,320.0,EXACT,4168.0,ACCEPTED,8631710.0,5.0,2568748.0,Xylaria hypoxylon,95.0,Sordariomycetes,False,"Xylaria hypoxylon (L.) Grev., 1824",Xylaria,Xylariales,1,60.0,NaN,NaN
3,4.jpg,4,4,Xylaria hypoxylon,http://mushroomobserver.org/images/320/4,1,2006-05-21 07:17:10,Fungi,Xylariaceae,8631710.0,SPECIES,Ascomycota,1298.0,Xylaria hypoxylon,98.0,320.0,EXACT,4168.0,ACCEPTED,8631710.0,5.0,2568748.0,Xylaria hypoxylon,95.0,Sordariomycetes,False,"Xylaria hypoxylon (L.) Grev., 1824",Xylaria,Xylariales,1,5.0,NaN,NaN
4,5.jpg,5,5,Xeromphalina,http://mushroomobserver.org/images/320/5,1,2006-05-21 07:17:12,Fungi,Mycenaceae,NaN,GENUS,Basidiomycota,1499.0,NaN,93.0,186.0,EXACT,4180.0,ACCEPTED,2527268.0,5.0,2527268.0,Xeromphalina,34.0,Agaricomycetes,False,Xeromphalina Kühner & Maire,Xeromphalina,Agaricales,1,36.0,NaN,NaN


In [54]:
# Suppression des colonnes non exploitées :
del_col = [
            'image_id',              # Id donné à une image, il s'agit du même nom que celui attribué au fichier, sans l'extension.
            'observation',           # Id donnant u nordre à l'observation, il est similaire à l'index et à image_id
            'user',                  # Id donné à un utilisateur, probablement ayant soumis l'observation
            'gbif_info/confidence',  # Pourcentage, basé sur un vote communautaire, pour classifier l'observation
            'gbif_info/matchType',   # ?
            'gbif_info/status',      # Statut de l'observation, si elle est acceptée ou rejetée sur le site. (une image de voiture serait refusée)
            'gbif_info/synonym',     # Booléen sans utilité pour nos analyses
            'thumbnail',             # ?
            'location',              # Flottant donnant un lieu de localisation. Non utilisé dnas notre analyse
            'gbif_info/note',        # Colonne de notes, composée intégralement de NaN
            'gbif_info'              # Colonnes d'informations, composée intégralement de NaN
          ]

mushroom_observation.drop(del_col,
                          axis = 1,
                          inplace = True)


print("Nombre de colonnes restantes (initial : 33) :", mushroom_observation.shape[1])

Nombre de colonnes restantes (initial : 33) : 22


In [55]:
# Traitement des doublons
mushroom_observation.drop_duplicates(inplace = True)
print("Nombre de dupliqués :", mushroom_observation.duplicated().sum())
print("Nombre d'observations restantes :", mushroom_observation.shape[0])

Nombre de dupliqués : 0
Nombre d'observations restantes : 647604


In [56]:
# Retrait du prefixe 'gbif_info/' dans le nom des colonnes
supp_prefixe_col = {
    'gbif_info/kingdom' : 'kingdom',
    'gbif_info/family' : 'family',
    'gbif_info/rank' : 'rank',
    'gbif_info/phylum' : 'phylum',
    'gbif_info/species' : 'species',
    'gbif_info/canonicalName' : 'canonicalName',
    'gbif_info/class' : 'class',
    'gbif_info/scientificName' : 'scientificName',
    'gbif_info/genus' : 'genus',
    'gbif_info/order' : 'order',
    'gbif_info/speciesKey' : 'speciesKey',
    'gbif_info/orderKey': 'orderKey',
    'gbif_info/classKey': 'classKey',
    'gbif_info/familyKey' : 'familyKey',
    'gbif_info/usageKey' : 'scientificNameKey',
    'gbif_info/kingdomKey' : 'kingdomKey',
    'gbif_info/genusKey' : 'genusKey',
    'gbif_info/phylumKey': 'phylumKey'
    }

mushroom_observation = mushroom_observation.rename(columns = supp_prefixe_col)

In [57]:
mushroom_observation.head()

,image_lien,label,image_url,date,kingdom,family,speciesKey,rank,phylum,orderKey,species,classKey,familyKey,scientificNameKey,kingdomKey,genusKey,canonicalName,phylumKey,class,scientificName,genus,order
0,1.jpg,Xylaria polymorpha,http://mushroomobserver.org/images/320/1,2006-05-21 07:17:05,Fungi,Xylariaceae,5255104.0,SPECIES,Ascomycota,1298.0,Xylaria polymorpha,320.0,4168.0,5255104.0,5.0,2568748.0,Xylaria polymorpha,95.0,Sordariomycetes,"Xylaria polymorpha (Pers.) Grev., 1824",Xylaria,Xylariales
1,2.jpg,Xylaria magnoliae,http://mushroomobserver.org/images/320/2,2006-05-21 07:17:06,Fungi,Xylariaceae,3461845.0,SPECIES,Ascomycota,1298.0,Xylaria magnoliae,320.0,4168.0,3461845.0,5.0,2568748.0,Xylaria magnoliae,95.0,Sordariomycetes,"Xylaria magnoliae J.D. Rogers, 1979",Xylaria,Xylariales
2,3.jpg,Xylaria hypoxylon,http://mushroomobserver.org/images/320/3,2006-05-21 07:17:08,Fungi,Xylariaceae,8631710.0,SPECIES,Ascomycota,1298.0,Xylaria hypoxylon,320.0,4168.0,8631710.0,5.0,2568748.0,Xylaria hypoxylon,95.0,Sordariomycetes,"Xylaria hypoxylon (L.) Grev., 1824",Xylaria,Xylariales
3,4.jpg,Xylaria hypoxylon,http://mushroomobserver.org/images/320/4,2006-05-21 07:17:10,Fungi,Xylariaceae,8631710.0,SPECIES,Ascomycota,1298.0,Xylaria hypoxylon,320.0,4168.0,8631710.0,5.0,2568748.0,Xylaria hypoxylon,95.0,Sordariomycetes,"Xylaria hypoxylon (L.) Grev., 1824",Xylaria,Xylariales
4,5.jpg,Xeromphalina,http://mushroomobserver.org/images/320/5,2006-05-21 07:17:12,Fungi,Mycenaceae,NaN,GENUS,Basidiomycota,1499.0,NaN,186.0,4180.0,2527268.0,5.0,2527268.0,Xeromphalina,34.0,Agaricomycetes,Xeromphalina Kühner & Maire,Xeromphalina,Agaricales


In [58]:
# Traitement des NaN
print("Nombre de NaN dans 'rank' :", mushroom_observation['rank'].isna().sum())


# Nous supprimons les lignes contenant des NaN dans la variable 'rank'.
mushroom_observation = mushroom_observation.dropna(subset=['rank'])
print("Valeurs restantes dans 'rank' :", mushroom_observation['rank'].unique())
print("Nombre d'observations restantes :", mushroom_observation.shape[0])

Nombre de NaN dans 'rank' : 33874
Valeurs restantes dans 'rank' : ['SPECIES' 'GENUS' 'ORDER' 'FORM' 'VARIETY' 'FAMILY' 'SUBSPECIES' 'PHYLUM'
 'KINGDOM' 'CLASS']
Nombre d'observations restantes : 613730


In [59]:
# Création d'une colonne 'saison'
print("Nombre de NaN dans 'date' :", mushroom_observation['date'].isna().sum())


# Ajout d'une colonne tampon 'date2' avec suppression des hh:mm:ss de la colonne 'date' :
mushroom_observation['date2'] = mushroom_observation['date'].str.slice(start=5, stop=7)

# Construction d'une fonction retournant la saison en fonction du mois observé
def get_season(date):
    if date == '12' or date == '01' or date == '02':
        return 'hiver'
    elif date == '03' or date == '04' or date == '05':
        return 'printemps'
    elif date == '06' or date == '07' or date == '08':
        return 'ete'
    else:
        return 'automne'
    
# Construction de la colonne 'saison' en appliquant la fonction 'get_season' sur la colonne 'date'
mushroom_observation['saison'] = mushroom_observation['date2'].apply(get_season)

# Suppression de la colonne 'date2' :
mushroom_observation.drop('date2', axis = 1, inplace = True)

Nombre de NaN dans 'date' : 0


In [80]:
# La variable rank représente le niveau de précision de classification de l'observation.
# Traitement des rank < 'family'

mushroom_observation_cleaned_sorted_ranks = mushroom_observation.loc[~mushroom_observation['rank'] \
                                    .isin(['KINGDOM', 'PHYLUM', 'CLASS', 'ORDER', 'VARIETY', 'FORM', 'SUBSPECIES'])]

print("Nombre d'observations restantes :", mushroom_observation_cleaned_sorted_ranks.shape[0])

Nombre d'observations restantes : 579313


In [79]:
# Sauvegarde du fichier de données nettoyé :
mushroom_observation.to_csv('../data/observations_mushroom_cleaned.csv', index=False)
mushroom_observation_cleaned_sorted_ranks.to_csv('../data/observations_mushroom_cleaned_sorted_rank.csv', index=False)

# Construction jeux de données

### mushroom_observation_key ne contenant que les variables 'key'

In [74]:
# Construction du dataset 
observations_mushroom_key = mushroom_observation[['image_lien',
                                                 'speciesKey',
                                                 'orderKey',
                                                 'classKey',
                                                 'scientificNameKey',
                                                 'familyKey',
                                                 'kingdomKey',
                                                 'genusKey',
                                                 'phylumKey']]

print("Nombre d'observations :", observations_mushroom_key.shape)
observations_mushroom_key.head()

Nombre d'observations : (613730, 9)


,image_lien,speciesKey,orderKey,classKey,scientificNameKey,familyKey,kingdomKey,genusKey,phylumKey
0,1.jpg,5255104.0,1298.0,320.0,5255104.0,4168.0,5.0,2568748.0,95.0
1,2.jpg,3461845.0,1298.0,320.0,3461845.0,4168.0,5.0,2568748.0,95.0
2,3.jpg,8631710.0,1298.0,320.0,8631710.0,4168.0,5.0,2568748.0,95.0
3,4.jpg,8631710.0,1298.0,320.0,8631710.0,4168.0,5.0,2568748.0,95.0
4,5.jpg,NaN,1499.0,186.0,2527268.0,4180.0,5.0,2527268.0,34.0


In [75]:
# Suppression des NaN
observations_mushroom_key = observations_mushroom_key.dropna()
print("Nombre d'observations :", observations_mushroom_key.shape)
observations_mushroom_key.isna().sum()

Nombre d'observations : (406852, 9)


image_lien           0
speciesKey           0
orderKey             0
classKey             0
scientificNameKey    0
familyKey            0
kingdomKey           0
genusKey             0
phylumKey            0
dtype: int64

In [77]:
# Enregistrement du dataset
observations_mushroom_key.to_csv('../data/observations_mushroom_key.csv', index=False)

### model_datas pour l'entrainement du modèle

In [64]:
temp_df = mushroom_observation_cleaned_sorted_ranks

Valeurs restantes dans 'rank' : ['SPECIES' 'GENUS' 'FAMILY' 'SUBSPECIES']
Nombre d'observations restantes : 580206


In [65]:
# Suppression de 'image_url' (sera reconstruite)
temp_df.drop('image_url', axis = 1, inplace = True)
temp_df.head()


# Fonction pour créer le chemin complet des images
def ajout_url(image_lien):
    filepath = "../../images/"
    return os.path.join(filepath, image_lien)

temp_df["image_url"] = temp_df["image_lien"].apply(ajout_url)

C:\Users\guill\AppData\Local\Temp\ipykernel_16196\372191355.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.drop('image_url', axis = 1, inplace = True)
C:\Users\guill\AppData\Local\Temp\ipykernel_16196\372191355.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["image_url"] = temp_df["image_lien"].apply(ajout_url)


In [82]:
# Retrait des colonnes non necessaires
model_data = temp_df[['label','image_lien', 'image_url', 'saison']]

In [83]:
# Suppression des doublons
model_data.drop_duplicates(inplace=True)
model_data.shape[0]

C:\Users\guill\AppData\Local\Temp\ipykernel_16196\2996696009.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data.drop_duplicates(inplace=True)


580082

In [84]:
# On ne garde que les 10 espèces les plus représentées
nb_val = model_data['label'].value_counts()
nb_val_label = nb_val.head(10)
nb_val_label = nb_val_label.index

model_data_10 = model_data[model_data['label'].isin(nb_val_label)]
model_data_10.reset_index(inplace=True, drop=True)

In [85]:
# l'image '611.56.jpg' n'existe pas, la ligne est donc supprimée du DataFrame pour ne pas créer d'erreur lors de l'entrainement
model_data_10 = model_data_10.drop(model_data_10[model_data_10['image_url'] == '../../images/611356.jpg'].index)
print("Nombre d'observations restantes :", model_data_10.shape[0])

Nombre d'observations restantes : 53092


In [86]:
model_data_10['label'].value_counts()

Russula        8235
Cortinarius    7352
Inocybe        5603
Psathyrella    5564
Mycena         5342
Amanita        4987
Agaricus       4692
Entoloma       4209
Gymnopilus     3744
Pluteus        3364
Name: label, dtype: int64

In [87]:
# On retire toutes les lignes comportant des images qui ont été retirées du dataset (images poubelles)

# img_supp.csv contient les fichiers images retirés du jeu de données inital
tri_img = pd.read_csv('../data/img_supp.csv', index_col = 0)
tri_img['fichiers'] = '../../images/' + tri_img['fichiers']
liste_supp = tri_img['fichiers'].tolist()
tri_img.head()

,fichiers
0,../../images/100403.jpg
1,../../images/101142.jpg
2,../../images/101143.jpg
3,../../images/101785.jpg
4,../../images/101786.jpg


In [88]:
indices_a_supprimer = model_data_10['image_url'].isin(liste_supp)
model_data_10_filtered = model_data_10[~indices_a_supprimer]
print("Nombre d'observations restantes :", model_data_10_filtered.shape[0])

Nombre d'observations restantes : 49861


In [89]:
model_data_10_filtered['label'].value_counts()

Russula        7946
Cortinarius    6970
Psathyrella    5124
Mycena         4928
Amanita        4887
Agaricus       4545
Inocybe        4435
Entoloma       3918
Gymnopilus     3744
Pluteus        3364
Name: label, dtype: int64

In [93]:
# Enregistrement des csv
model_data_10.to_csv('../data/model_data_10.csv', index=False)
model_data_10_filtered.to_csv('../data/model_data_10_filtered.csv', index=False)